# Convolutional Neural Network Example

In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd

### Load the MNIST Data

In [2]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

In [4]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

### Architect the model

In [5]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [6]:
cnn = CNN()
cnn.cuda()

CNN (
  (layer1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (fc): Linear (1568 -> 10)
)

In [7]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

### Model Training

In [8]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Iter [100/600] Loss: 0.1166
Epoch [1/5], Iter [200/600] Loss: 0.1105
Epoch [1/5], Iter [300/600] Loss: 0.0424
Epoch [1/5], Iter [400/600] Loss: 0.0328
Epoch [1/5], Iter [500/600] Loss: 0.0892
Epoch [1/5], Iter [600/600] Loss: 0.0301
Epoch [2/5], Iter [100/600] Loss: 0.0435
Epoch [2/5], Iter [200/600] Loss: 0.0311
Epoch [2/5], Iter [300/600] Loss: 0.0132
Epoch [2/5], Iter [400/600] Loss: 0.0393
Epoch [2/5], Iter [500/600] Loss: 0.0435
Epoch [2/5], Iter [600/600] Loss: 0.0153
Epoch [3/5], Iter [100/600] Loss: 0.0083
Epoch [3/5], Iter [200/600] Loss: 0.0208
Epoch [3/5], Iter [300/600] Loss: 0.0304
Epoch [3/5], Iter [400/600] Loss: 0.0240
Epoch [3/5], Iter [500/600] Loss: 0.0188
Epoch [3/5], Iter [600/600] Loss: 0.0350
Epoch [4/5], Iter [100/600] Loss: 0.0313
Epoch [4/5], Iter [200/600] Loss: 0.0273
Epoch [4/5], Iter [300/600] Loss: 0.0129
Epoch [4/5], Iter [400/600] Loss: 0.0205
Epoch [4/5], Iter [500/600] Loss: 0.0612
Epoch [4/5], Iter [600/600] Loss: 0.0508
Epoch [5/5], Ite

## Inference (Test the model)

In [42]:
my_labels = []
my_predictions = []

# Test the Model
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    my_labels += labels.tolist()
    my_predictions += predicted.cpu().tolist()

In [57]:
print('Test Accuracy of the model on the 10000 test images: %d %%\n' % (100 * correct / total))
print('%d out of %d classified correctly' % (correct, total))
# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

Test Accuracy of the model on the 10000 test images: 98 %

9861 out of 10000 classified correctly


In [49]:
df = pd.DataFrame({})
df['label'] = my_labels
df['prediction'] = my_predictions

In [52]:
df[df['label'] != df['prediction']].head(15)

,label,prediction
18,3,8
92,9,4
320,9,8
321,2,7
359,9,4
445,6,0
449,3,5
492,2,8
583,2,7
619,1,8
